In [1]:
%pip install graphviz
%pip install pygraphviz

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pygraphviz as pgv
import os

def method_cout(dot_file_path):
    output_file_path = os.path.splitext(dot_file_path)[0] + "_methods.txt"
    graph = pgv.AGraph(dot_file_path)
    methods = []

    # Traverse the nodes in the graph
    for node in graph.nodes():
        label = node.attr['label']
        if label:
            # Extract method information from the label
            lines = label.split('\\n')
            if len(lines) > 1:
                method_name = lines[0].strip()
                metrics_line = lines[1].strip()
                if method_name and 'min:' in metrics_line:
                    metrics = {}
                    metrics_parts = [part for part in metrics_line.split(',') if part.strip()]
                    for part in metrics_parts:
                        key, value = part.split(':')
                        metrics[key.strip()] = value.strip()
                    methods.append({
                        'method': method_name,
                        'metrics': metrics
                    })
    
    # Output the extracted methods
    # for method in methods:
    #     print(f"Method: {method['method']}")
    #     for metric, value in method['metrics'].items():
    #         print(f"  {metric}: {value}")
    #     print()
    # Write the extracted methods to the output file
    with open(output_file_path, 'w') as output_file:
        output_file.write(f"Total number of methods in the dot file are:{len(methods)}\n")
        output_file.write("Extracted Methods:\n\n")
        for method in methods:
            output_file.write(f"Method: {method['method']}\n")
            for metric, value in method['metrics'].items():
                output_file.write(f"  {metric}: {value}\n")
            output_file.write("\n")




#dot_file = '/Users/fabiha/Documents/mono2micro/mono2micro/jpet-6-master/account_tracelog/assemblyOperationDependencyGraph.dot'
#/Users/fabiha/Documents/mono2micro/mono2micro/jpet-6-master/category_tracelog/assemblyOperationDependencyGraph.dot
#/Users/fabiha/Documents/mono2micro/mono2micro/jpet-6-master/order_tracelog/assemblyOperationDependencyGraph.dot
# Load the DOT file
dot_file_path = '/Users/fabiha/Documents/mono2micro/mono2micro/jpet-6-master/order_tracelog/assemblyOperationDependencyGraph.dot'
method_cout(dot_file_path)


In [8]:
import pygraphviz as pgv
import os
import re

def extract_methods(dot_file_path):
    graph = pgv.AGraph(dot_file_path)
    methods = []

    # Traverse the subgraphs to get class names
    for subgraph in graph.subgraphs():
        class_label = subgraph.graph_attr['label']
        class_match = re.search(r'@[\d]+:(.*)', class_label)
        if class_match:
            class_name = class_match.group(1).strip()
            # Traverse the nodes in the subgraph
            for node in subgraph.nodes():
                label = node.attr['label']
                if label:
                    # Extract method information from the label
                    lines = label.split('\\n')
                    if len(lines) > 1:
                        method_name = lines[0].strip()
                        metrics_line = lines[1].strip()
                        if method_name and 'min:' in metrics_line:
                            metrics = {}
                            metrics_parts = [part for part in metrics_line.split(',') if part.strip()]
                            for part in metrics_parts:
                                key, value = part.split(':')
                                metrics[key.strip()] = value.strip()
                            methods.append({
                                'class': class_name,
                                'method': method_name,
                                'metrics': metrics
                            })

    return methods

def count_unique_methods(dot_file_paths):
    unique_methods = {}
    
    for dot_file_path in dot_file_paths:
        methods = extract_methods(dot_file_path)
        for method in methods:
            unique_key = (method['class'], method['method'])
            if unique_key not in unique_methods:
                unique_methods[unique_key] = method['metrics']
    
    return unique_methods

def save_methods_to_file(unique_methods, output_file_path):
    with open(output_file_path, 'w') as output_file:
        output_file.write(f"Total number of unique methods in the dot files: {len(unique_methods)}\n")
        output_file.write("Extracted Methods:\n\n")
        for (class_name, method_name), metrics in unique_methods.items():
            output_file.write(f"Class: {class_name}\n")
            output_file.write(f"Method: {method_name}\n")
            for metric, value in metrics.items():
                output_file.write(f"  {metric}: {value}\n")
            output_file.write("\n")

# Specify the paths to the DOT files
dot_file_paths = [
    '/Users/fabiha/Documents/mono2micro/mono2micro/jpet-6-master/account_tracelog/assemblyOperationDependencyGraph.dot',
    '/Users/fabiha/Documents/mono2micro/mono2micro/jpet-6-master/category_tracelog/assemblyOperationDependencyGraph.dot',
    '/Users/fabiha/Documents/mono2micro/mono2micro/jpet-6-master/order_tracelog/assemblyOperationDependencyGraph.dot'
]

# Count unique methods from the specified DOT files
unique_methods = count_unique_methods(dot_file_paths)

# Specify the output file path
output_file_path = '/Users/fabiha/Documents/mono2micro/mono2micro/jpet-6-master/dynamic_analysis_covered_methods.txt'

# Save the unique methods to the output file
save_methods_to_file(unique_methods, output_file_path)

print(f"Unique methods have been saved to {output_file_path}")


Unique methods have been saved to /Users/fabiha/Documents/mono2micro/mono2micro/jpet-6-master/dynamic_analysis_covered_methods.txt
